In [1]:
#Importing necessary Libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
#Loading Dataset
#data = pd.read_csv("raw_data_ff6.csv")
data = pd.read_csv("all_data.csv")

In [3]:
data.pop('rms2')#not needed in training 
data.pop('mean2')#not needed in training 
data.pop('kurtosis2')#not needed in training 
data.pop('SD2')#not needed in training 
data.pop('skewness2')#not needed in training 

0       0.262897
1       0.263335
2       0.263561
3       0.269052
4       0.267861
          ...   
3245    0.469787
3246    0.472097
3247    0.474144
3248    0.481275
3249    0.481073
Name: skewness2, Length: 3250, dtype: float64

In [4]:
data.head()

,rms,mean,kurtosis,SD,skewness,Class
0,44.810008,40.643678,-0.802444,18.882279,-0.388448,0
1,44.905504,40.739943,-0.790491,18.901712,-0.385872,0
2,44.877419,40.683908,-0.797525,18.955702,-0.385073,0
3,44.903024,40.729885,-0.787285,18.917512,-0.387942,0
4,44.919868,40.742816,-0.789494,18.929668,-0.387397,0


In [5]:
#Generating Matrix of Features
X = data.iloc[:,0:-1].values
print(X.shape)

(3250, 5)


In [6]:
#Generating Dependent Variable Vectors
Y = data.iloc[:,-1].values

In [7]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=0)

In [8]:
#Performing Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
#Initialising ANN
ann = tf.keras.models.Sequential()
#Adding First Hidden Layer
ann.add(tf.keras.layers.Dense(units=100,activation="relu"))
#Adding Second Hidden Layer
ann.add(tf.keras.layers.Dense(units=50,activation="relu"))
#Adding Third Hidden Layer
ann.add(tf.keras.layers.Dense(units=10,activation="relu"))
#Adding Output Layer
ann.add(tf.keras.layers.Dense(units=6,activation="softmax"))

2023-02-07 14:22:54.875701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
#Compiling ANN
ann.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])

In [11]:
#Fitting ANN
ann.fit(X_train, Y_train, verbose=1, epochs=45) #epoch 45 gives highest accuracy

Epoch 1/45
72/72 [==============================] - 0s 1ms/step - loss: 1.7105 - accuracy: 0.3125
Epoch 2/45
72/72 [==============================] - 0s 879us/step - loss: 1.5711 - accuracy: 0.3552
Epoch 3/45
72/72 [==============================] - 0s 885us/step - loss: 1.4700 - accuracy: 0.3780
Epoch 4/45
72/72 [==============================] - 0s 976us/step - loss: 1.3946 - accuracy: 0.4180
Epoch 5/45
72/72 [==============================] - 0s 1ms/step - loss: 1.3391 - accuracy: 0.4435
Epoch 6/45
72/72 [==============================] - 0s 859us/step - loss: 1.2974 - accuracy: 0.4554
Epoch 7/45
72/72 [==============================] - 0s 858us/step - loss: 1.2675 - accuracy: 0.4615
Epoch 8/45
72/72 [==============================] - 0s 1ms/step - loss: 1.2373 - accuracy: 0.4747
Epoch 9/45
72/72 [==============================] - 0s 939us/step - loss: 1.2144 - accuracy: 0.4884
Epoch 10/45
72/72 [==============================] - 0s 881us/step - loss: 1.1860 - accuracy: 0.5125
Epoch

In [12]:
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               600       
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 dense_3 (Dense)             (None, 6)                 66        
                                                                 
Total params: 6,226
Trainable params: 6,226
Non-trainable params: 0
_________________________________________________________________


In [13]:
probabilities = ann.predict(X_test)
Y_pred = []
for i in range(len(probabilities)):
    max = 0
    for j in range(6):
        if(probabilities[i][j]>probabilities[i][max]):
            max = j
    Y_pred.append(max)

print(Y_pred)
print(len(Y_pred))
from sklearn.metrics import accuracy_score 
accuracy_score(Y_test, Y_pred)

31/31 [==============================] - 0s 706us/step
[2, 0, 1, 0, 0, 5, 1, 2, 1, 4, 5, 0, 1, 4, 5, 2, 0, 1, 5, 0, 1, 5, 1, 2, 1, 0, 5, 5, 4, 1, 4, 5, 1, 4, 4, 1, 1, 4, 2, 0, 1, 4, 4, 5, 1, 0, 5, 0, 2, 2, 1, 1, 1, 5, 2, 2, 5, 1, 2, 2, 4, 0, 5, 2, 2, 4, 2, 0, 5, 0, 5, 0, 4, 1, 0, 4, 2, 2, 4, 1, 1, 0, 0, 1, 0, 1, 0, 5, 2, 2, 0, 5, 2, 5, 1, 4, 0, 4, 2, 2, 0, 0, 4, 0, 0, 0, 4, 1, 4, 0, 4, 4, 0, 0, 5, 1, 1, 0, 0, 5, 2, 4, 1, 5, 4, 2, 0, 5, 4, 4, 2, 0, 1, 1, 2, 0, 2, 4, 1, 2, 0, 2, 4, 5, 0, 5, 2, 2, 0, 2, 1, 1, 0, 4, 1, 2, 2, 1, 1, 1, 1, 0, 4, 0, 0, 1, 1, 4, 0, 0, 1, 4, 1, 2, 4, 4, 1, 2, 2, 2, 5, 0, 0, 2, 1, 2, 0, 4, 0, 5, 1, 0, 4, 5, 2, 2, 2, 4, 1, 1, 5, 5, 5, 1, 1, 0, 0, 2, 5, 5, 2, 2, 2, 4, 1, 2, 5, 1, 0, 2, 0, 4, 4, 5, 4, 5, 2, 0, 0, 5, 4, 2, 5, 4, 2, 2, 1, 2, 1, 1, 5, 2, 0, 2, 1, 2, 4, 2, 2, 0, 4, 4, 0, 2, 4, 5, 5, 2, 1, 4, 5, 1, 4, 2, 2, 5, 5, 1, 4, 2, 0, 0, 4, 5, 2, 2, 4, 2, 5, 0, 2, 0, 5, 5, 0, 0, 4, 5, 1, 2, 0, 2, 0, 1, 2, 4, 4, 1, 2, 0, 2, 1, 4, 5, 1, 0, 2, 0, 0, 0, 0, 2, 4, 4, 2,

0.7630769230769231